<a href="https://colab.research.google.com/github/nehaal10/Image_captioning/blob/main/genset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import pandas as pd
import shutil
from nltk import corpus
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
import re
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pickle
from keras.preprocessing.text import Tokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
os.chdir('/content/drive/MyDrive/kaggle')

path=os.getcwd()

os.environ['KAGGLE_CONFIG_DIR'] = path

!kaggle datasets download -d hsankesara/flickr-image-dataset

100% 8.16G/8.16G [03:20<00:00, 59.8MB/s]
100% 8.16G/8.16G [03:20<00:00, 43.7MB/s]


In [ ]:
os.mkdir('/content/drive/MyDrive/project')
os.chdir('/content/drive/MyDrive/project')
shutil.move('/content/drive/MyDrive/kaggle/flickr-image-dataset.zip',os.getcwd())

!unzip -q 'flickr-image-dataset.zip'

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/project/flickr30k_images/results.csv',error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 3: expected 1 fields, saw 2\nSkipping line 22: expected 1 fields, saw 4\nSkipping line 39: expected 1 fields, saw 2\nSkipping line 48: expected 1 fields, saw 2\nSkipping line 68: expected 1 fields, saw 2\nSkipping line 92: expected 1 fields, saw 4\nSkipping line 93: expected 1 fields, saw 3\nSkipping line 94: expected 1 fields, saw 2\nSkipping line 97: expected 1 fields, saw 3\nSkipping line 99: expected 1 fields, saw 2\nSkipping line 101: expected 1 fields, saw 2\nSkipping line 117: expected 1 fields, saw 2\nSkipping line 133: expected 1 fields, saw 2\nSkipping line 187: expected 1 fields, saw 3\nSkipping line 202: expected 1 fields, saw 3\nSkipping line 208: expected 1 fields, saw 2\nSkipping line 222: expected 1 fields, saw 2\nSkippin

In [ ]:
def create_new_dataframe(data):
  image_name=[]
  count=[]
  description=[]
  for i in range(len(data)):
    text=data.iloc[i][0]
    text=text.split()
    image_name.append(text[0].strip('|'))
    if '|' not in text[1]:
      count.append(text[1])
    else:
      txt=text[1].strip('|')
      count.append(txt)

    line=' '.join(text[2:])
    description.append(line)


  all_df={'image_name':image_name,'count':count,'description':description}

  df=pd.DataFrame(all_df)

  return df

df=create_new_dataframe(data)

In [ ]:
def clean_text(data):
  for i in range(len(data)):
    txt=data.loc[i,'description']
    txt=txt.lower()
    txt=re.sub(r'[^a-zA-Z0-9]',' ',txt)
    #txt=nltk.word_tokenize(txt)
    #stop=set(stopwords.words('english'))
    #txt=[w for w in txt if w not in stop]
    #lem=WordNetLemmatizer()
    #txt=[lem.lemmatize(j) for j in txt]

    data.loc[i,'description']=txt


clean_text(df)

In [ ]:
def deleting_missing(data):
  name=list(data['image_name'].unique())
  path='/content/drive/MyDrive/project/flickr30k_images/flickr30k_images'
  name_folder=os.listdir(path)

  for i in name_folder:
    if i not in name:
      image_path=path+'/'+i
      if os.path.isfile(image_path) is True:
        os.remove(image_path)
      else:
        shutil.rmtree(image_path)


deleting_missing(df)


In [ ]:
def recising(image_path):
  dim=(224,224)
  os.chdir(image_path)
  for i in os.listdir(image_path):
    img=cv2.imread(i)
    img_resized= cv2.resize(img,dim) 
    img=cv2.cvtColor(img_resized,cv2.COLOR_BGR2RGB)
    os.remove(i)
    cv2.imwrite(i,img)


recising('/content/drive/MyDrive/project/flickr30k_images/flickr30k_images')

In [3]:
df=pd.read_csv('/content/drive/MyDrive/project/flickr30k_images/new.csv')

In [4]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df

,image_name,count,description
0,1000092795.jpg,0,two young guys with shaggy hair look at their ...
1,1000092795.jpg,2,two men in green shirts are standing in a yard
2,1000092795.jpg,3,a man in a blue shirt standing in a garden
3,1000092795.jpg,4,two friends enjoy time spent together
4,10002456.jpg,0,several men in hard hats are operating a giant...
...,...,...,...
141846,99804383.jpg,2,a older asian man is playing an instrument in ...
141847,99804383.jpg,3,an elderly man sits outside a storefront accom...
141848,99804383.jpg,4,an elderly gentleman playing a musical instrum...
141849,998845445.jpg,2,a man is leaning off of the side of a blue and...


--------

In [5]:
# spliting the data set
df.drop_duplicates(subset='image_name',keep='first',inplace=True)
train_df,test_df=train_test_split(df,test_size=0.3,shuffle=True,random_state=42)

val_df,test_df=train_test_split(test_df,test_size=0.2,random_state=43)

train_df=train_df.reset_index()
train_df.drop('index',axis=1,inplace=True)

val_df=val_df.reset_index()
val_df.drop('index',axis=1,inplace=True)

In [6]:
training_images_path=[]
val_images_path=[]
training_captions=list(train_df['description'].apply(lambda x: x.strip()))
val_captions=list(val_df['description'].apply(lambda x: x.strip()))

path='/content/drive/MyDrive/project/flickr30k_images/flickr30k_images/'
for i in range(len(train_df)):
  img=train_df.loc[i,'image_name']
  img=path+img
  training_images_path.append(img)

for i in range(len(val_df)):
  image=val_df.loc[i,'image_name']
  image=path+image
  val_images_path.append(image)



# **processing it to tensorflow**

In [23]:
def load_img(image_path):
  pass

img=tf.io.read_file('/content/drive/MyDrive/project/flickr30k_images/flickr30k_images/2757779501.jpg')
img=tf.io.decode_jpeg(img,channels=3)
img.shape

TensorShape([224, 224, 3])

In [22]:
training_images_path[:5]

['/content/drive/MyDrive/project/flickr30k_images/flickr30k_images/2757779501.jpg',
 '/content/drive/MyDrive/project/flickr30k_images/flickr30k_images/3862675867.jpg',
 '/content/drive/MyDrive/project/flickr30k_images/flickr30k_images/3628564816.jpg',
 '/content/drive/MyDrive/project/flickr30k_images/flickr30k_images/374828031.jpg',
 '/content/drive/MyDrive/project/flickr30k_images/flickr30k_images/544257613.jpg']

In [8]:
import collections
img_to_cap_vector = collections.defaultdict(list)

In [9]:
img_to_cap_vector

defaultdict(list, {})

------------------------------------------------------

In [10]:
# get features process text and build vocab for training
len(df['image_name'].unique()),len(os.listdir('/content/drive/MyDrive/project/flickr30k_images/flickr30k_images'))

(31780, 31780)

In [12]:

def feature_extracter(df,mode='train'):
  os.chdir('/content/drive/MyDrive/project/flickr30k_images/flickr30k_images')

  model=tf.keras.applications.vgg16.VGG16(include_top=True)

  model.layers.pop()

  cnn=tf.keras.Model(inputs=model.inputs, outputs=model.layers[-1].output)

  for i in range(len(df)):
    img=df.loc[i,'image_name']
    img=Image.open(img)
    img=tf.keras.utils.img_to_array(img)
    img=img.reshape((1,img.shape[0],img.shape[1],img.shape[2]))
    f=cnn.predict(img)
    if 'train' in mode:
      features_train.append(f)
      captions_train.append(df['description'][i])
    elif 'val' in mode:
      features_val.append(f)
      captions_val.append(df['description'][i])


features_train=[]
captions_train=[]
features_val=[]
captions_val=[]

df.drop_duplicates(subset='image_name',keep='first',inplace=True)

train_df,test_df=train_test_split(df,test_size=0.3,shuffle=True,random_state=42)

val_df,test_df=train_test_split(test_df,test_size=0.2,random_state=43)

train_df=train_df.reset_index()
train_df.drop('index',axis=1,inplace=True)

val_df=val_df.reset_index()
val_df.drop('index',axis=1,inplace=True)



feature_extracter(train_df,'train')
print('Train Over')
feature_extracter(val_df,'val')
print('val Over')

553476096/553467096 [==============================] - 3s 0us/step
Train Over
val Over


In [13]:
##

def save_data():
  path='/content/drive/MyDrive/project/flickr30k_images/'
  
  with open(path+'features_train.txt','wb') as f:
    pickle.dump(features_train,f)

  with open(path+'captions_train.txt','wb') as g:
    pickle.dump(captions_train,g)

  with open(path+'features_val.txt','wb') as h:
    pickle.dump(features_val,h)

  with open(path+'captions_val.txt','wb') as d:
    pickle.dump(captions_val,d)






In [ ]:
with open('/content/drive/MyDrive/project/flickr30k_images/captions_train.txt','rb') as f :
  captions_train=pickle.load(f)

with open('/content/drive/MyDrive/project/flickr30k_images/captions_val.txt','rb') as t:
  captions_val=pickle.load(t)

with open('/content/drive/MyDrive/project/flickr30k_images/features_train.txt','rb') as f:
  features_train=pickle.load(f)

with open('/content/drive/MyDrive/project/flickr30k_images/features_val.txt','rb') as g:
  features_val=pickle.load(g)


In [14]:
len(captions_train),len(features_train),len(captions_val),len(features_val)

(22246, 22246, 7627, 7627)

In [18]:
features_train[0][0]

array([1.75753954e-08, 4.49038922e-08, 5.34693413e-07, 1.71636557e-07,
       9.23234040e-07, 1.76350582e-07, 7.20099024e-07, 4.28017131e-07,
       2.08154731e-07, 3.39748027e-07, 3.08342543e-08, 4.99729076e-08,
       1.07295911e-07, 1.49356964e-07, 2.43516808e-07, 2.59384507e-07,
       8.76365434e-08, 9.00892019e-08, 7.21140111e-07, 1.34071570e-07,
       6.96978077e-07, 1.66438696e-07, 2.20918707e-07, 7.24179108e-06,
       1.34005731e-07, 1.09449354e-06, 9.19211587e-08, 1.74311276e-06,
       3.77436251e-08, 1.85056948e-08, 1.71495742e-08, 6.10452418e-08,
       4.21902229e-08, 1.87613159e-06, 3.36433664e-06, 2.42048401e-07,
       1.22458050e-06, 1.78508841e-07, 2.08405808e-07, 2.56177213e-06,
       4.69831605e-07, 1.93029905e-07, 3.24978396e-06, 6.14516722e-08,
       3.43084764e-07, 8.07215565e-06, 5.75090041e-07, 3.59489221e-08,
       4.66118217e-06, 3.18983581e-07, 1.84441603e-07, 1.09751841e-06,
       8.41351323e-07, 1.28862581e-07, 6.35320305e-07, 3.97310259e-07,
      

In [ ]:
tk=Tokenizer()
tk.fit_on_texts(captions_train)
word_index=tk.word_index
encoded_caption=tk.texts_to_sequences(captions_train)
start=len(word_index)+1
stop=start+1
vocab=stop+1

encoded_captions = [([start] + c) for c in encoded_caption]



In [ ]:
end

NameError: ignored

In [ ]:
print(encoded_caption[0])
encoded_captions[0]


[329, 40, 61, 262, 470, 2329]


[8502, 329, 40, 61, 262, 470, 2329]

In [ ]:
cap=[]
for i in captions_train:
  word=nltk.word_tokenize(i)
  cap.append(word)

cap=tf.constant(cap)
layer=tf.keras.layers.StringLookup()
layer.adapt(cap)
len(layer.get_vocabulary())

ValueError: ignored

In [ ]:
c='three shopper looking product table outside market'

c.replace(' ','\n')

'three\nshopper\nlooking\nproduct\ntable\noutside\nmarket'

In [19]:
import shutil
shutil.rmtree('/content/drive/MyDrive/project/flickr30k_images/flickr30k_images')